#### Normal Workflow Application (Rule-Based / Traditional AI)

Everything follows predefined rules and fixed steps.

Step-by-step flow

1. User places order
2. System checks inventory
3. System calculates price + tax
4. Payment gateway is called
5. If payment = success → confirm order
6. If payment = failure → show error
7. Generate invoice
8. Notify warehouse

#### Agentic AI Applications

Instead of following a fixed script, the AI agent thinks, plans, and acts.

What the Agent does internally

1. Understand the goal
      + Goal: Resolve customer issue & retain customer
2. Decide what actions are needed
      + Check order status
      + Check warehouse location
      + Check courier delay
      + Check refund/compensation policies
3. Choose tools dynamically
      + Order DB tool
      + Logistics API
      + Email / chat tool
      + Coupon system
4. Act autonomously
      + Expedite shipping OR
      + Offer partial refund OR
      + Offer coupon OR
      + Escalate to human
5. Learn
      + Did customer accept?
      + Was resolution successful?

##### What is an Agent?

An Agent is an AI system that can reason about a goal and take actions (using tools) to achieve it.

Reasoning vs Action 

1. Reasoning (Thinking)
     + Understanding the problem
     + Breaking it into steps
     + Deciding which tool to use
     + Planning the next move
2. Action (Doing)
     + Calling APIs
     + Searching databases
     + Running code
     + Sending emails
     + Querying documents

##### Agent Loop
Reason >> Act >> Observe >> Reason >> Act >> …

##### ReAct (Reason + Act)
ReAct Pattern
Thought → Action → Observation → Thought → Action → Final Answer

Query = “What is the population of the capital of Karnataka?”

ReAct Flow:

1. Thought: Capital of Karnataka is Bangalore.
2. Thought: I need population data.
3. Action: search_tool("Bangalore population")
4. Observation: Population is ~12 million.
5. Thought: I have the answer.
6. Final Answer: Bangalore has a population of about 12 million.

##### Tool-Calling Behavior

Tools are external capabilities an agent can use.

Examples:

+ Search engines
+ Databases
+ APIs
+ Calculators
+ File systems
+ Email services

In [23]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

load_dotenv()
assert os.getenv("OPENAI_API_KEY")

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_react_agent(
    model=llm,
    tools=[multiply]
)

result = agent.invoke({
    "messages": [("user", "What is 37 * 48?")]
})

print("✅ Final Answer:", result["messages"][-1].content)

print("Raw model output:\n", result) 


C:\Users\LotusBlue\AppData\Local\Temp\ipykernel_35140\2824844466.py:18: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


✅ Final Answer: The result of \( 37 \times 48 \) is 1776.
Raw model output:
 {'messages': [HumanMessage(content='What is 37 * 48?', additional_kwargs={}, response_metadata={}, id='3ddd8628-53cd-4654-82a1-a716a0cf5cb9'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 52, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CpgyaqTBXdfKUBT2VkY5lGimy7N6s', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b47c9-c523-7830-9aca-0372e00f4bb6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 37, 'b': 48}, 'id': 'call_WhwzskV9YxAkID4gCklzo1Qb', 'type': 'tool_call'}], usage_

In [11]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# ReAct agent runner (prebuilt, most stable in modern ecosystem)
from langgraph.prebuilt import create_react_agent

load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found in env!"

# --- Tools ---
@tool
def calc(expression: str) -> str:
    """Evaluate a simple arithmetic expression like '12*7 + 3'."""
    # NOTE: For demo only. In production, use a safe math parser.
    return str(eval(expression))

@tool
def product_policy(query: str) -> str:
    """Lookup store policy (mock tool)."""
    policies = {
        "return policy": "Returns accepted within 7 days with bill and tags intact.",
        "refund timeline": "Refunds take 3-5 business days after pickup verification.",
        "exchange policy": "Exchange allowed once within 7 days for size/color issues."
    }
    # naive matching for demo
    for k, v in policies.items():
        if k in query.lower():
            return v
    return "Policy not found. Ask support team."

tools = [calc, product_policy]

# --- LLM ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- Create ReAct Agent ---
agent = create_react_agent(llm, tools)

# --- Run a query that needs tools ---
query = """
A customer says: "I bought a saree for 2499 and got 12% discount.
What is the final price? Also tell me the return policy."
"""

result = agent.invoke({"messages": [("user", query)]})

# Final answer is in the last message content
print(result["messages"][-1].content)


C:\Users\LotusBlue\AppData\Local\Temp\ipykernel_35140\236767696.py:40: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


The final price of the saree after a 12% discount is ₹2199.12. 

Regarding the return policy, returns are accepted within 7 days, provided the bill and tags are intact.
